In [1]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/02 14:34:12 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/02 14:34:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/02 14:34:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Converting Crash Data to parquet file

In [4]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.parquet"
df.write.parquet(parquet_output_path)


23/08/02 14:35:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/NYC_Vehicle_Collisions_Crashes.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

REMOVING CRASHES WITH NULL FOR BOROUGH (LOCATION)

In [64]:
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("Remove Null Rows").getOrCreate()

# Read the parquet file into a DataFrame
Crash_results = spark.read.parquet('../data/raw/NYC_Vehicle_Collisions_Crashes.parquet')

# Filter out rows with null values in the "BOROUGH" column
Crash_results_filtered = Crash_results.na.drop(subset=["BOROUGH"])

# Show the filtered DataFrame
Crash_results_filtered.show()

# Save the filtered DataFrame to a new Parquet file
Crash_results_filtered.write.parquet('../data/landing/Crash_results_filtered.parquet', mode='overwrite')

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME| CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBU

Convert Crime data csv's to parquets

In [65]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/shooting_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/shooting_data.parquet"
df.write.parquet(parquet_output_path)

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/shooting_data.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [10]:
from pyspark.sql import SparkSession

# Assuming you already have a SparkSession called "spark"
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")

# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/complaint_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/complaint_data.parquet"
df.write.parquet(parquet_output_path)

In [12]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/arrest_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/arrest_data.parquet"
df.write.parquet(parquet_output_path)

Removing unneccesary columns from crime data

Shooting Data

In [3]:
from pyspark.sql import SparkSession

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Shot_Data").getOrCreate()

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/shooting_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Select the columns you want to keep and rename them
selected_columns = [
    "INCIDENT_KEY as ID",
    "OCCUR_DATE as Date",
    "BORO as Borough",
    "PRECINCT as Precinct",
    "Latitude",
    "Longitude"
]
df_selected = df.selectExpr(*selected_columns)

# Step 4: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/shooting_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

# Stop the SparkSession
spark.stop()


Complaint Data

In [4]:
from pyspark.sql import SparkSession

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Complaint_Data").getOrCreate()

# Set the configuration to rebase INT96 datetime values during writing
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")  # or "CORRECTED" depending on your use case

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/complaint_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Select the columns you want to keep and rename them
selected_columns = [
    "CMPLNT_NUM as ID",
    "CMPLNT_FR_DT as Date",
    "BORO_NM as Borough",
    "ADDR_PCT_CD as Precinct",
    "Latitude",
    "Longitude"
]
df_selected = df.selectExpr(*selected_columns)

# Step 4: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/complaint_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

df_selected.show()

# Stop the SparkSession
spark.stop()


+--------+-------------------+---------+--------+----------------+-----------------+
|      ID|               Date|  Borough|Precinct|        Latitude|        Longitude|
+--------+-------------------+---------+--------+----------------+-----------------+
|10600119|2002-01-01 11:00:00|MANHATTAN|    null|            null|             null|
|11052575|2005-04-22 10:00:00|    BRONX|    null|            null|             null|
|10832306|2005-07-11 10:00:00| BROOKLYN|    null|            null|             null|
|10107192|2005-10-19 10:00:00|    BRONX|    null|            null|             null|
|23893731|2005-12-04 11:00:00|    BRONX|    null|            null|             null|
|10049834|2006-01-09 11:00:00|    BRONX|    null|            null|             null|
|10142316|2006-01-11 11:00:00|MANHATTAN|    null|            null|             null|
|10097651|2006-01-13 11:00:00|   QUEENS|    null|            null|             null|
|10107697|2006-01-15 11:00:00|    BRONX|    null|            null

Arrest Data

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Arrest_Data").getOrCreate()

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/arrest_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Define a mapping dictionary for boroughs
borough_mapping = {
    "B": "Bronx",
    "S": "Staten Island",
    "K": "Brooklyn",
    "M": "Manhattan",
    "Q": "Queens"
}

# Step 4: Select the columns you want to keep and rename them, while mapping borough abbreviations
df_selected = df.select(
    col("ARREST_KEY").alias("ID"),
    col("ARREST_DATE").alias("Date"),
    when(col("ARREST_BORO").isin(list(borough_mapping.keys())), col("ARREST_BORO")).otherwise(col("ARREST_BORO")).alias("Borough"),
    col("ARREST_PRECINCT").alias("Precinct"),
    col("Latitude"),
    col("Longitude")
).replace(borough_mapping, subset=["Borough"])

# Step 5: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/arrest_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

# Step 6: Show the resulting DataFrame
df_selected.show()

# Step 7: Stop the SparkSession
spark.stop()



23/08/03 09:21:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+---------+--------+------------------+------------------+
|       ID|               Date|  Borough|Precinct|          Latitude|         Longitude|
+---------+-------------------+---------+--------+------------------+------------------+
|236791704|2021-11-22 11:00:00|Manhattan|      28|40.799008797000056|-73.95240854099995|
|237354740|2021-12-04 11:00:00|    Bronx|      41|40.816391847000034|-73.89529641399997|
|236081433|2021-11-09 11:00:00|   Queens|     113| 40.67970040800003|-73.77604736799998|
| 32311380|2007-06-18 10:00:00|   Queens|      27|              null|              null|
|192799737|2019-01-26 11:00:00|Manhattan|      25|40.800694331000045|-73.94110928599997|
|193260691|2019-02-06 11:00:00|Manhattan|      14| 40.75783900300007|-73.99121211099998|
|237291769|2021-12-03 11:00:00|   Queens|     115| 40.77205649600006|-73.87622400099998|
|236106641|2021-11-10 11:00:00|    Bronx|      41|40.804012949000025|-73.87833183299993|
|238383628|2021-12-28

Mapping abbreviations of Borough to their full names in Arrest Data 

Merging all crime data

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Merge_Parquet_Files").getOrCreate()

# Set the configuration for legacy mode
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")

# Step 2: Define the file paths for all the Parquet files you want to merge
file_paths = [
    "../data/raw/arrest_data_reduced.parquet",
    "../data/raw/shooting_data_reduced.parquet",
    "../data/raw/complaint_data_reduced.parquet",
    # Add more file paths as needed
]

# Step 3: Read each Parquet file into a separate DataFrame
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Step 4: Add a new column to each DataFrame to identify the source (file name)
source_column_name = "Source"
for i in range(len(dfs)):
    dfs[i] = dfs[i].withColumn(source_column_name, lit(file_paths[i]))

# Step 5: Concatenate or union the DataFrames to create a single DataFrame
merged_df = dfs[0]
for i in range(1, len(dfs)):
    merged_df = merged_df.union(dfs[i])

# Step 6: Write the merged DataFrame to a new Parquet file
parquet_output_path = "../data/raw/merged_crime_data.parquet"
merged_df.write.parquet(parquet_output_path)

merged_df.show()

# Stop the SparkSession
spark.stop()


+---------+-------------------+---------+--------+------------------+------------------+--------------------+
|       ID|               Date|  Borough|Precinct|          Latitude|         Longitude|              Source|
+---------+-------------------+---------+--------+------------------+------------------+--------------------+
|236791704|2021-11-22 11:00:00|Manhattan|      28|40.799008797000056|-73.95240854099995|../data/raw/arres...|
|237354740|2021-12-04 11:00:00|    Bronx|      41|40.816391847000034|-73.89529641399997|../data/raw/arres...|
|236081433|2021-11-09 11:00:00|   Queens|     113| 40.67970040800003|-73.77604736799998|../data/raw/arres...|
| 32311380|2007-06-18 10:00:00|   Queens|      27|              null|              null|../data/raw/arres...|
|192799737|2019-01-26 11:00:00|Manhattan|      25|40.800694331000045|-73.94110928599997|../data/raw/arres...|
|193260691|2019-02-06 11:00:00|Manhattan|      14| 40.75783900300007|-73.99121211099998|../data/raw/arres...|
|237291769